In [1]:
from sagemaker.pytorch import PyTorchModel
import sagemaker
import boto3
from datetime import datetime
import time

In [2]:
s3_client = boto3.client('s3')
session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [3]:
!tar -czvf model.tar.gz code model.pt
print("Model data ready")

Model data ready


a code
a code/backbone.py
a code/BEATs.py
a code/inference.py
a code/modules.py
a code/quantizer.py
a code/README.md
a code/requirements.txt
a code/Tokenizers.py
a model.pt


In [4]:
bucket_name = 'beats-finetuning'
model_file = 'model.tar.gz'

In [5]:
s3_client.delete_object(Bucket=bucket_name, Key=model_file)
s3_client.upload_file(model_file, bucket_name, model_file)

In [6]:
model_name = "model.pt"
model_data = f"s3://{bucket_name}/model.tar.gz"

In [7]:
model = PyTorchModel(entry_point = "inference.py",
py_version = "py3",
model_data = model_data,
env = {"MODEL_NAME": model_name},
role = role,
sagemaker_session = session
)

In [8]:
INSTANCE_TYPE = "ml.m5.4xlarge"
ENDPOINT_NAME = "beats-pytorch-" + str(datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S-%f"))

In [9]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
    content_type = 'audio/wav'
)

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not access model data at s3://sagemaker-us-east-1-410677554255/pytorch-inference-2023-08-31-21-11-14-933/model.tar.gz. Please ensure that the role "arn:aws:iam::410677554255:role/aws-reserved/sso.amazonaws.com/AWSReservedSSO_SANDBOX_ADM_80911213bcf8fa12" exists and that its trust relationship policy allows the action "sts:AssumeRole" for the service principal "sagemaker.amazonaws.com". Also ensure that the role has "s3:GetObject" permissions and that the object is located in us-east-1. If your Model uses multiple models or uncompressed models, please ensure that the role has "s3:ListBucket" permission.

In [ ]:
audio_file = 'baby.wav'
with open(audio_file, 'rb') as f:
    audio_bytestream = f.read()

start_time = time.time()
predictions = predictor.predict(audio_bytestream)
duration = time.time() - start_time

print(predictions)
print(duration)

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)